# Imports

In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix

import numpy as np

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, LeaveOneOut, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score

import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
py.init_notebook_mode(connected=True)

import pprint
pp = pprint.PrettyPrinter(indent=2)

from collections import Counter, OrderedDict

# About the dataset 

In [ ]:
#Read some information about the dataset from its text description file
filereader = open("WineDataset/winequality.names", "r")
for txt_line in filereader:
    print(txt_line)
filereader.close()

# Read the dataset

## Red Wine

In [ ]:
#Read red wine dataset
red_dataset = pd.read_csv('WineDataset/winequality-red.csv', sep=";")

In [ ]:
#Print some information about the red wine dataset imported.
# number of entries (1599), coloumns description (missing attribute values?, type value description..), memory usage
red_dataset.info()

In [ ]:
# Have a short view of red wine dataset
red_dataset.head(5)

In [ ]:
# Generate descriptive statistics for coloumns (count, mean, std, min and max, lower percentile, median, 
# and upper percentile)
red_dataset.describe()

In [ ]:
# Let's analyze the target variable (quality)
# From dataset description quality value has a between 0 and 10... let's see how it is distributed.
print(np.sort(np.array(red_dataset['quality'].unique())))
print(dict(OrderedDict(sorted(dict(Counter(red_dataset['quality'])).items()))))

## White Wine

In [ ]:
# Repeat the previous steps this time with the white wine dataset
# Read white wine dataset
white_dataset = pd.read_csv('WineDataset/winequality-white.csv', sep=";")

In [ ]:
#Print some information about the white wine dataset imported.
# number of entries (4898), coloumns description (missing attribute values?, type value description..), memory usage
white_dataset.info()
# --> the columns are equivalent to red wine dataset

In [ ]:
# Have a short view of white wine dataset
white_dataset.head(5)

In [ ]:
# Generate descriptive statistics for coloumns (count, mean, std, min and max, lower percentile, median, 
# and upper percentile)
white_dataset.describe()

In [ ]:
# Let's analyze the target variable (quality)
# From dataset description quality value has a between 0 and 10... let's see how it is distributed.
print(np.sort(np.array(white_dataset['quality'].unique())))
print(dict(OrderedDict(sorted(dict(Counter(white_dataset['quality'])).items()))))
# --> more open range (9 wine quality) than red wine dataset but same distribution

# Correlation between features and datasets

In [ ]:
# Let's study the correlation between features for each dataset and then and then we compare the correlations 
# between the two datasets.
# To evaluate the linear correlation between two variables has been used the Pearson correlation coefficient. 
# It has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and 
# −1 is total negative linear correlation.

headers = list(red_dataset.columns.values)
allCorrCoefRed = []
allHoverTextRed = []
allCorrCoefWhite = []
allHoverTextWhite = []

for i in range(len(headers)):
    corrCoefRed = []
    hoverTextRed = []
    corrCoefWhite = []
    hoverTextWhite = []
    for j in range(len(headers)):
        if (j>i):
            corrCoefRed.append(None)
            hoverTextRed.append(None)
            corrCoefWhite.append(None)
            hoverTextWhite.append(None)
            break
        red = np.corrcoef(red_dataset[headers[i]].values, red_dataset[headers[j]].values)[1,0]
        white = np.corrcoef(white_dataset[headers[i]].values, white_dataset[headers[j]].values)[1,0]
        hoverTextRed.append("Correlation between '" + headers[i] + "' and '" + headers[j] + "' is: " + str(round(red,2)));
        corrCoefRed.append(red)
        hoverTextWhite.append("Correlation between '" + headers[i] + "' and '" + headers[j] + "' is: " + str(round(white,2)));
        corrCoefWhite.append(white)
    allCorrCoefRed.append(corrCoefRed)
    allHoverTextRed.append(hoverTextRed)
    allCorrCoefWhite.append(corrCoefWhite)
    allHoverTextWhite.append(hoverTextWhite)
    
data = [go.Heatmap(
            z = allCorrCoefRed,
            x = headers,
            y = headers,
            hoverinfo = "text",
            hovertext = allHoverTextRed,
            colorscale='IceFire')]
layout = go.Layout(title='<b>Red Wine</b> Correlation between all features and quality.')
py.iplot(go.Figure(data, layout))

data = [go.Heatmap(
            z = allCorrCoefWhite,
            x = headers,
            y = headers,
            hoverinfo = "text",
            hovertext = allHoverTextWhite,
            colorscale='IceFire')]
layout = go.Layout(title='<b>White Wine</b> Correlation between all features and quality.')
py.iplot(go.Figure(data, layout))

In [ ]:
# Let's compare correlation between the two dataset to to test the possibility of conducting a study that 
# includes both datasets. 

qualityCorrCoefRed = dict()
for i in range(len(allCorrCoefRed[11])-1):
    qualityCorrCoefRed[headers[i]] = allCorrCoefRed[11][i] 
qualityCorrCoefRed = dict(sorted(qualityCorrCoefRed.items(), key=lambda kv: abs(kv[1]), reverse=True))

qualityCorrCoefWhite = dict()
for i in range(len(allCorrCoefWhite[11])-1):
    qualityCorrCoefWhite[headers[i]] = allCorrCoefWhite[11][i] 
qualityCorrCoefWhite = dict(sorted(qualityCorrCoefWhite.items(), key=lambda kv: abs(kv[1]), reverse=True))

data = [
    go.Bar(name='Red', 
            x=list(qualityCorrCoefRed.values()), 
            y=list(qualityCorrCoefRed.keys()), 
            orientation='h',
            marker_color='#AC1E44'),
    go.Bar(name='White', 
            x=list(qualityCorrCoefWhite.values()), 
            y=list(qualityCorrCoefWhite.keys()), 
            orientation='h',
            marker_color='#aee1e5')
]
layout = go.Layout(title='<b>Red and White Wine</b> Correlation between features and <i>quality</i> (target).', barmode='group')
py.iplot(go.Figure(data, layout))

# The correlations will result to be similar in some cases but totally opposite in others with the conclusion
# that it is better to conduct two separate studies for white and red wines

# Red Wine

In [ ]:
# Focus on the red wine dataset for this study
# Let's produce an overiew on each feature and target (quality) correlation to have a visual representation of the
# correlation and the presence of outliers.

features = list(red_dataset)
target = features[-1]
features = features[:-1]

fig = make_subplots(
    rows = 4, 
    cols = 3,
    horizontal_spacing = 0.08,
    vertical_spacing = 0.03,
)

r = c = 1
for feature in features:
    fig.add_trace(
        go.Box(
            y = red_dataset[feature], 
            x=red_dataset[target],
            boxpoints='outliers',
            jitter=0.3,
            marker=dict(
                size=5,
                opacity=0.5,
                color='rgb(8,81,156)',
                outliercolor='rgb(219, 64, 82)',
            ),
            line_color='rgb(8,81,156)',
            line_width=1,
            showlegend=False
        ),
        row = r, col = c)
    fig.update_yaxes(title_text=feature, ticks="inside", nticks=7, row=r, col=c)
    fig.update_xaxes(ticks="inside", tick0=1, dtick=1, row=r, col=c)
    if (r == 4):
        fig.update_xaxes(title_text=target, row=r, col=c)
    if (r == 3 & c == 3):
        fig.update_xaxes(title_text=target, row=r, col=c)
    c = c + 1
    if (c > 3):
        c = 1
        r = r + 1
        
fig.update_layout(title_text="BoxPlot overview of features and target correlation with outliers check.", 
                  height=1000, 
                  width=1000)
fig.show()

# Boxplots show many outliers for quite a few columns. The 'red_dataset.describe()' table output produced some 
# cells above explain analytically this fact.
# Some features as residual sugar, chlorides and sulphates present an huge gap between min and max value. This
# could explain the outliers. 
# In some features a linear correlation is observed more than in others (alcohol, volatile acidity, sulphates...)

# REGRESSION

In [ ]:
# Some functions...
def dataEstimateAndRegressionLine(w0, w1, feature, featureName, target, targetName):
    x_line = np.array([np.min(feature), np.max(feature)])
    y_line = w0 + w1*x_line
    data = [
        go.Scatter(
            x=feature, 
            y=target, 
            mode="markers", 
            marker_color = "#000000",
            marker_opacity = 0.7,
            marker_line_width=1, 
            marker_size=8,
            marker_line_color = "#ffffff",
            name="data"
        ),
        go.Scatter(
            x=feature, 
            y=y_hat, 
            mode="markers", 
            marker_color = "#007fff",
            marker_opacity = 0.7,
            marker_line_width=1, 
            marker_size=8,
            marker_line_color = "#ffffff",
            name="estimate"
        ),
        go.Scatter(
            x=x_line, 
            y=y_line, 
            mode="lines", 
            name="regression line"
        )
    ]
    for i in range(len(feature)):
        data.append(
            go.Scatter(
                x=[feature[i], feature[i]], 
                y=[target[i], y_hat[i]], 
                mode="lines",
                showlegend=False, 
                line=dict(color="gray", width=0.5)
            )
        )
    layout = go.Layout(
        xaxis = dict(title=featureName),
        yaxis = dict(title=targetName)
    )
    return data, layout

In [ ]:
def residualPlotAndHistogram(feature, featureName, target, y_hat):
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(
        go.Scatter(
            x = feature, 
            y = target - y_hat, 
            mode = "markers",
            marker_color = "#007fff",
            marker_opacity = 0.7,
            marker_line_width = 1, 
            marker_size = 8,
            marker_line_color = "#ffffff",
            showlegend = False
        ),
        row = 1, col = 1
    )
    fig.add_trace(
        go.Histogram(
            y=target - y_hat, 
            showlegend = False,
            marker = dict(
                color = "#007fff",
            ),
        ),
        row = 1, col = 2
    )
    fig.update_xaxes(title_text = featureName, row = 1, col = 1)
    fig.update_yaxes(title_text = "{} Residual".format(featureName), row = 1, col = 1)
    fig.update_xaxes(title_text = "Number of instances", row = 1, col = 2)
    fig.update_layout(
        title = 'Plot and histogram of the residual'
    )
    return fig

In [ ]:
def computePerformance(y, y_hat):       
    err = (y-y_hat)**2 
    SSE = np.sum(err)
    errMean = (y-np.mean(y))**2
    R2 = 1 - (SSE/( np.sum(errMean))) 
    return SSE, R2

In [ ]:
def AdjustedRSquare(R2, n, p):
    # n is number of observations in sample
    # p is number of independent variables in model
    adjr2 = 1-(1-R2)*(n-1)/(n-p-1)
    return adjr2

In [ ]:
def goodnessIndicators(SSE, R2, title, text1, text2):
    fig = go.Figure()
    fig.add_trace(go.Indicator(
        title = text1,
        mode = 'number',
        value = SSE,
        domain = {'row': 0, 'column': 0}))
    fig.add_trace(go.Indicator(
        title = text2,
        mode = 'number',
        value = R2,
        domain = {'row': 0, 'column': 1}))
    fig.update_layout(
        height = 300,
        title = title,
        grid = {'rows': 1, 'columns': 2, 'pattern': "independent"}
    )
    return fig

In [ ]:
def accuracyScoreIndicator(score):
    fig = go.Figure(
        go.Indicator(
            title = 'Accuracy Score',
            mode = 'number',
            value = score,
            )
    )
    fig.update_layout(
        height = 300
    )
    return fig

In [ ]:
def t(x):
    print(type(x))
    print(x.shape)

## Univariate linear regression (alcohol)

In [ ]:
# Let's start modelling with a simple univariate linear regression using alcohol feature and quality target.
feature = red_dataset["alcohol"].values
target = red_dataset["quality"].values
feature_reshape = np.reshape(feature,(len(feature),1))
# Build the model
linReg = LinearRegression()
linReg.fit(feature_reshape, target)
w0 = linReg.intercept_
w1 = linReg.coef_
# Predict
y_hat = linReg.predict(feature_reshape)

In [ ]:
# Graphically represent the data and the regression line
data, layout = dataEstimateAndRegressionLine(w0, w1, feature, "Alcohol", target, "Quality")
py.iplot(go.Figure(data, layout))

In [ ]:
# Graphically represent the residual
fig = residualPlotAndHistogram(feature, "Alcohol", target, y_hat)
fig.show()

In [ ]:
# Take a look at the score of the model
# SSE stands for Sum of Squared Errors
# R square is the percentage of the response variable variation that is explained by a linear model [0-1]
SSE, R2 = computePerformance(target,y_hat)
mse = mean_squared_error(target,y_hat)
fig = goodnessIndicators(mse, R2, "Assessing goodness...", "MSE", "R\u00b2")
fig.show()

# ... high SSE and low model explanation

## Univariate linear regression (volatile acidity)

In [ ]:
# Another simple univariate linear regression using volatile acidity feature and quality target.

feature = red_dataset["volatile acidity"].values
feature_reshape = np.reshape(feature,(len(feature),1))
# Build the model
linReg = LinearRegression()
linReg.fit(feature_reshape, target)
# Predict
y_hat = linReg.predict(feature_reshape)

In [ ]:
# Take a look at the score of the model
SSE, R2 = computePerformance(target,y_hat)
mse = mean_squared_error(target,y_hat)
fig = goodnessIndicators(mse, R2, "Assessing goodness...", "MSE", "R\u00b2")
fig.show()

# ... higher SSE then previous model and even low model explanation (as expected by correlation study done previously)

## Simple Multivariate linear regression (alcohol + volatile acidity)

In [ ]:
# Let's modelling now with a simple multivariate linear regression using alcohol and volatile acidity feature 
# and quality target.

features = red_dataset[['alcohol', 'volatile acidity']].values
target = red_dataset['quality']

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.20, random_state=0)
# Build the model
linReg = LinearRegression()
linReg.fit(xTrain, yTrain)
w = linReg.coef_
w0 = linReg.intercept_
# Take a look at w values
print(w)
print(w0)

In [ ]:
# Graphically represent the data and the 3D plane
x_line = [np.min(xTrain[:,0]), np.max(xTrain[:,0])]
y_line = [np.min(xTrain[:,1]), np.max(xTrain[:,1])]
z_line = np.zeros((2,2))

for ind1 in range(2):
    for ind2 in range(2):
        xpred = np.array([ [x_line[ind1], y_line[ind2] ] ]) 
        z_line[ind1,ind2] = linReg.predict(xpred)


trace = [go.Scatter3d(x = xTrain[:,0], y = xTrain[:,1], z = target, mode = 'markers', marker = dict(size = 3), name = 'measured data' ),
         go.Surface(x = x_line, y = y_line, z = z_line, name = 'linear regression')]
layout = go.Layout(scene = dict(xaxis = dict(title = 'Alcohol'), 
                                yaxis = dict(title = 'Volatile Acidity'), 
                                zaxis = dict(title = 'Quality')),
                  title = '3D Scatter plot with regression plan (Training Set). <b>Alcohol</b> + <b>Volatile Acidity</b> features, <b>Quality</b> target')

fig = go.Figure(trace, layout)
py.iplot(fig)

In [ ]:
# Predict
yhatTrain = linReg.predict(xTrain)
yhatTest = linReg.predict(xTest)
# Evaluate the model for training set
R2Train = r2_score(yTrain, yhatTrain)
adjR2Train = AdjustedRSquare(R2Train, len(xTrain), len(xTrain[0]))
mseTrain = mean_squared_error(yTrain, yhatTrain)
# Evaluate the model for test set
R2Test = r2_score(yTest, yhatTest)
adjR2Test = AdjustedRSquare(R2Test, len(xTest), len(xTest[0]))
mseTest = mean_squared_error(yTest, yhatTest)

In [ ]:
# Take a look at the score of the model
fig = goodnessIndicators(
    mseTrain, 
    adjR2Train, 
    "Assessing goodness in Training set:", "MSE", "Adjusted R\u00b2")
fig.show()
fig = goodnessIndicators(
    mseTest, 
    adjR2Test, 
    "Assessing goodness in Test set:", "MSE", "Adjusted R\u00b2")
fig.show()

# As expected, R squared increase adding more features to the model, even if they are unrelated to the response. 
# Selecting the model with the highest R-squared is not a reliable approach for choosing the best linear model.
# In this 'goodness assessing' I have used the Adjusted R squared, more suitable for multivariate linear regression

## Multivariate linear regression with StratifiedKFold Validation

In [ ]:
# For a better approach to feature selection cross-validation provides a more reliable estimate of out-of-sample 
# error, and thus is a better way to choose which of your models will best generalize to out-of-sample data.

# Stratified K-Folds cross-validator provides train/test indices to split data in train/test sets. This 
# cross-validation object is a variation of KFold that returns stratified folds. The folds are made by 
# preserving the percentage of samples for each class.
# --> This is relevant to our case in which the target variable is not uniformly distributed

featuresList = list(red_dataset)
targett = featuresList[-1]
featuresList = featuresList[:-1]
features = red_dataset[featuresList].values
target = red_dataset[targett].values

# Generate indices fro cross-validation
skf = StratifiedKFold(n_splits = 10) #The least populated class in target (wine with quality=3) has only 10 members. 
skf.get_n_splits(features)

# In this study I used MSE, which stands for mean squared errors and Adjusted R square for evaluate the regression 
# model

MSETrainAll = [] 
MSETestAll = []
AdjustedR2TrainAll = []
AdjustedR2TestAll = []

for train_index, test_index in skf.split(features, target):
    xTrain, xTest = features[train_index], features[test_index]
    yTrain, yTest = target[train_index], target[test_index]
    # Build the regressor
    model = LinearRegression()
    # Fit
    model.fit(xTrain, yTrain)
    # Predict
    yhatTrain = model.predict(xTrain)
    yhatTest = model.predict(xTest)
    # Evaluate the model for training set
    R2Train = r2_score(yTrain, yhatTrain)
    adjR2Train = AdjustedRSquare(R2Train, len(xTrain), len(xTrain[0]))
    mseTrain = mean_squared_error(yTrain, yhatTrain)
    # Evaluate the model for test set
    R2Test = r2_score(yTest, yhatTest)
    adjR2Test = AdjustedRSquare(R2Test, len(xTest), len(xTest[0]))
    mseTest = mean_squared_error(yTest, yhatTest)
    # Collect values for final mean assessing
    MSETrainAll.append(mseTrain)
    MSETestAll.append(mseTest)
    AdjustedR2TrainAll.append(adjR2Train)
    AdjustedR2TestAll.append(adjR2Test)
# Present the result by calculating the average of single cross-validation result
fig = goodnessIndicators(
    np.mean(MSETrainAll), 
    np.mean(AdjustedR2TrainAll), 
    "Assessing goodness in Training set:", "MSE", "Adjusted R\u00b2")
fig.show()
fig = goodnessIndicators(
    np.mean(MSETestAll), 
    np.mean(AdjustedR2TestAll), 
    "Assessing goodness in Test set:", "MSE", "Adjusted R\u00b2")
fig.show()

## Multivariate linear regression with Ridge and Lasso Regolarization Analysis

In [ ]:
# Let's work now with Ridge and Lasso regolarization maintaining Stratified K-Folds cross-validator as the 
# previous study to be able to evaluate a possible improvement.
# The regolarizations have been built with different alpha values taken from this set:
# 1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20, 1e2, 1e3
# Then the alhpa for each regolarizations that takes to the best score is used for the evaluation of the regressor

skf = StratifiedKFold(n_splits = 10) #The least populated class in target has only 10 members.
skf.get_n_splits(features)

wLassoAll = []
wRidgeAll = []
LassoMSETestAll = []
LassoAdjustedR2TestAll = []
RidgeMSETestAll = []
RidgeAdjustedR2TestAll = []
alphaValues = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20, 1e2, 1e3]
for alpha in alphaValues:

    wLasso = []
    wRidge = []
    scoreLasso = []
    scoreRidge = []
    mseLasso = []
    mseRidge = []
    
    for train_index, test_index in skf.split(features, target):
        xTrain, xTest = features[train_index], features[test_index]
        yTrain, yTest = target[train_index], target[test_index]

        model = Lasso(alpha=alpha, max_iter=10e5)
        model.fit(xTrain, yTrain)
        w = model.coef_
        wLasso.append(w)
        yhatTest = model.predict(xTest)
        R2Test = r2_score(yTest, yhatTest)
        adjR2Test = AdjustedRSquare(R2Test, len(xTest), len(xTest[0]))
        mseTest = mean_squared_error(yTest, yhatTest)
        scoreLasso.append(adjR2Test)
        mseLasso.append(mseTest)
        
        model2 = Ridge(alpha=alpha, max_iter=10e5)
        model2.fit(xTrain, yTrain)
        w = model2.coef_
        wRidge.append(w)
        yhatTest = model2.predict(xTest)
        R2Test = r2_score(yTest, yhatTest)
        adjR2Test = AdjustedRSquare(R2Test, len(xTest), len(xTest[0]))
        mseTest = mean_squared_error(yTest, yhatTest)
        scoreRidge.append(adjR2Test)
        mseRidge.append(mseTest)
        
    
    LassoMSETestAll.append(np.mean(mseLasso))
    LassoAdjustedR2TestAll.append(np.mean(scoreLasso))
    RidgeMSETestAll.append(np.mean(mseRidge))
    RidgeAdjustedR2TestAll.append(np.mean(scoreRidge))
    
    wLassoAll.append(np.mean(wLasso, axis=0))
    wRidgeAll.append(np.mean(wRidge, axis=0))
    
lassoAlphaI = 0
maxScoreLasso = LassoAdjustedR2TestAll[0]
ridgeAlphaI = 0
maxScoreRidge = RidgeAdjustedR2TestAll[0]
i = 0
for score in LassoAdjustedR2TestAll:
    if (score > maxScoreLasso):
        lassoAlphaI = i
        maxScoreLasso = score
    i = i + 1
i = 0
for score in RidgeAdjustedR2TestAll:
    if (score > maxScoreRidge):
        ridgeAlphaI = i
        maxScoreRidge = score
    i = i + 1
# Print best alpha for both regolarizations
print("Best alpha for Ridge: " + str(alphaValues[ridgeAlphaI]))
print("Best alpha for Lasso: " + str(alphaValues[lassoAlphaI]))
# Take a look at the score of the model
fig = goodnessIndicators(
    LassoMSETestAll[lassoAlphaI],
    LassoAdjustedR2TestAll[lassoAlphaI],
    "<b>Ridge regolarization</b> assessing goodness in Test set:", "MSE", "Adjusted R\u00b2")
fig.show()
fig = goodnessIndicators(
    RidgeMSETestAll[ridgeAlphaI],
    RidgeAdjustedR2TestAll[ridgeAlphaI],
    "<b>Lasso regolarization</b> assessing goodness in Test set:", "MSE", "Adjusted R\u00b2")
fig.show()


# --> a slight improvement was made to the score but nothing considerable

In [ ]:
# Graphically represent the W coefficients for each regolarizations for each alpha value
def getW(i, coll):
    w = []
    for array in coll:
        w.append(array[i])
    return w

alphaValuesStr = []
for n in alphaValues:
    alphaValuesStr.append(str(n))

featuresList = list(red_dataset)
featuresList = featuresList[:-1]
wAll = [wRidgeAll, wLassoAll]

density = 0
i=0
for f in featuresList:
    if (f == "density"):
        density = i
    i = i + 1

titles = ["<b>Ridge regolarization</b> coefficients at the increase of alpha", 
          "<b>Lasso regolarization</b> coefficients at the increase of alpha"]
titI = 0
for ws in wAll:
    fig = go.Figure()
    for i in range(len(ws[0])):
        # Skip density w because of its huge value that ruin the graphics
        if (i == density): continue
        fig.add_trace(
            go.Scatter(
                x = list(range(len(alphaValues))), 
                y = getW(i, ws), 
                line_shape='spline',
                name = featuresList[i]
            )
        )
    fig.update_layout(
        title = titles[titI],
        height = 400
    ),
    titI = titI + 1
    fig.update_yaxes(
        title = "w",
    ),
    fig.update_xaxes(
        title = "Alpha",
        ticktext=alphaValuesStr,
        tickvals=list(range(len(ws[0])+1)),
    )
    fig.show()
    
# --> Remember: best alpha for Ridge regression was 1 and for Lasso 0.0001
# Ridge regolarization shows how the coefficient tend to zero at the same alpha values while Lasso takes all 
# coefficient to zero in an uncoordinated way (some are zero before others) that could be useul to perform 
# a feature selection analysis.

# CLASSIFICATION

In [ ]:
# Transform the problem into a classification one. We want now to classificate each istance of red wine, based
# on features, into three categories: Bad wine, Average wine and Good wine
BadAverageOrGood = []
for v in target:
    if (v <= 4):
        BadAverageOrGood.append(1)
        continue
    if (v <= 6):
        BadAverageOrGood.append(2)
        continue
    BadAverageOrGood.append(3)
counterForEachClass = dict(OrderedDict(sorted(dict(Counter(BadAverageOrGood)).items())))
print(counterForEachClass)
BadAverageOrGood = np.array(BadAverageOrGood)
percentForEachClass = dict()
for i in range(len(np.unique(BadAverageOrGood))):
    percentForEachClass[i+1] = counterForEachClass[i+1] / len(BadAverageOrGood)
print(percentForEachClass)

In [ ]:
features = red_dataset[list(red_dataset)[:-1]].values

# Before making any actual predictions, it is always a good practice to scale the features so that all of them 
# can be uniformly evaluated. Wikipedia explains the reasoning pretty well:
#      "Since the range of values of raw data varies widely, in some machine learning algorithms, objective 
#      functions will not work properly without normalization. For example, the majority of classifiers calculate 
#      the distance between two points by the Euclidean distance. If one of the features has a broad range of 
#      values, the distance will be governed by this particular feature. Therefore, the range of all features 
#      should be normalized so that each feature contributes approximately proportionately to the final distance."
sc = StandardScaler()
features = sc.fit_transform(features)

In [ ]:
# Let's start modelling. All model are evalueted using train and test set achieved slpitting the dataset in two parts,
# one for training set (80%) and the other for test set (20%). the stratified option in sklearn train_test_split
# keeps equal proportions of each class train/test set.
# Evaluation concerns accuracy and misclassification costs.

xTrain, xTest, yTrain, yTest = train_test_split(
    features, 
    BadAverageOrGood, 
    test_size=0.20, 
    stratify=BadAverageOrGood, 
    random_state=0)
misclassificationMatrix = [[0, 1, 5], [1, 0, 1], [5, 1, 0]]

# Some functions:
def evaluateClassificator(cm, out):
    df = pd.DataFrame(cm,
             index = [['','MEASURED',''], ['Bad', 'Average', 'Good']],
             columns = [['','PREDICTED',''],['Bad', 'Average', 'Good']])
    score = accuracy_score(yTest, yhatTest)
    cost = 0
    for i in range(len(cm)):
        for j in range(len(cm)):
            cost = cost + cm[i][j] * misclassificationMatrix[i][j]
    if out:
        print()
        print(df)
        print()
        print("Accuracy Score: " + str(round(score,3)) + "%")
        print()
        print("Misclassification cost: " + str(cost))
        return
    return score, cost

# Follow in order:
# • Logistic Regression
# • KNeighborsClassifier
# • Support Vector Classifier (SVC)
# • Gaussian Naive Bayes
# • Gaussian Process Classifier
# • Decision Tree Classifier
# • Random Forest Classifier

### Logistic Regression

In [ ]:
model = LogisticRegression(multi_class="auto", solver="liblinear", max_iter=10e5, random_state=0)
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)
print()
model = KNeighborsClassifier(n_neighbors=5)
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### Support Vector Classifier (SVC)

In [ ]:
model = SVC(gamma='auto')
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

In [ ]:
model = SVC()
param = {
    'C': [0.1,0.8,0.9,1,1.1,1.2,1.3,1.4],
    'kernel':['linear', 'rbf'],
    'gamma' :[0.1,0.8,0.9,1,1.1,1.2,1.3,1.4]
}
grid_svc = GridSearchCV(model, param_grid=param, scoring='accuracy', cv=10)
grid_svc.fit(xTrain, yTrain)
bestParams = grid_svc.best_params_
print("Best params: " + str(bestParams))
model = SVC(C = bestParams['C'], gamma =  bestParams['gamma'], kernel= bestParams['kernel'])
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### Gaussian Naive Bayes

In [ ]:
model = GaussianNB()
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### Gaussian Process Classifier

In [ ]:
model = GaussianProcessClassifier()
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### Decision Tree Classifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

### Random Forest Classifier

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)

In [ ]:
rf = RandomForestClassifier()
param = {
    'n_estimators' : [100],
    'max_depth' : np.linspace(1, 32, 32, endpoint=True),
    'min_samples_split' : np.linspace(0.1, 1.0, 10, endpoint=True),
    'min_samples_leaf' : np.linspace(0.1, 0.5, 5, endpoint=True),
    'max_features' : list(range(1, xTrain.shape[1]))
}
grid_rf = GridSearchCV(estimator = rf, param_grid = param, cv = 10, n_jobs = -1, verbose = 2)
# grid_rf.fit(xTrain, yTrain) (1 Hour of tuning)
# --> Best params: {'max_depth':1.0,'max_features':1,'min_samples_leaf':0.1,'min_samples_split':0.1,'n_estimators':100}
#bestParams = grid_rf.best_params_
bestParams = {
    'max_depth': 1.0, 
    'max_features': 1, 
    'min_samples_leaf': 0.1, 
    'min_samples_split': 0.1, 
    'n_estimators': 100
}

In [ ]:
rf = RandomForestClassifier(bestParams)
model.fit(xTrain, yTrain)
yhatTest = model.predict(xTest)
evaluateClassificator(confusion_matrix(yTest, yhatTest), True)